# Model I/O

## FewShotPromptTemplate

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

/Users/jaylee_83/Documents/_itsjayspace/git_clones/Langchain-7-day-challenge/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_72765/634784058.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [2]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}')

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Korea")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Greece?\nAI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Korea?'

In [5]:

chain = prompt | chat

chain.invoke({"country": "Spain"})

AI:  
I know this:  
Capital: Madrid  
Language: Spanish  
Food: Paella and Tapas  
Currency: Euro  

AIMessage(content='AI:  \nI know this:  \nCapital: Madrid  \nLanguage: Spanish  \nFood: Paella and Tapas  \nCurrency: Euro  ', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--7288dcb9-f8e8-4ea2-b797-11c4a66819dd-0')

## FewShotChatMessagePromptTemplate

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [8]:
example_prompt = ChatPromptTemplate.from_messages(
  [
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}"),
  ]
)

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you kjnow about {country}?")
  ]
)

In [9]:
chain= final_prompt | chat

chain.invoke(
  {
    "country" : "Korea"
  }
)


        I know this:
        Divided into two countries: South Korea and North Korea
        Capital of South Korea: Seoul
        Capital of North Korea: Pyongyang
        Languages: Korean
        Currency (South Korea): South Korean Won
        Currency (North Korea): North Korean Won
        

AIMessage(content='\n        I know this:\n        Divided into two countries: South Korea and North Korea\n        Capital of South Korea: Seoul\n        Capital of North Korea: Pyongyang\n        Languages: Korean\n        Currency (South Korea): South Korean Won\n        Currency (North Korea): North Korean Won\n        ', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--1f5b06d2-8d6d-48aa-a575-208a4e5c815d-0')

## LengthBasedExampleSelector

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
  ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [9]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
  examples=examples,
  example_prompt=example_prompt,
  max_length=80
)

In [12]:
prompt = FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"]
)

# prompt.format(country="Brazil")

In [13]:

chain = prompt | chat

chain.invoke({"country": "Spain"})

AI: Here is what I know about Spain:

- **Capital**: Madrid
- **Language**: Spanish (Castilian)
- **Food**: Tapas, paella, and jamón ibérico
- **Currency**: Euro
- **Culture**: Known for its vibrant festivals, flamenco music and dance, and rich history influenced by various cultures, including Roman and Moorish.
- **Geography**: Features diverse landscapes, including beaches, mountains, and plains, with notable regions like Catalonia, Andalusia, and the Basque Country.

AIMessage(content='AI: Here is what I know about Spain:\n\n- **Capital**: Madrid\n- **Language**: Spanish (Castilian)\n- **Food**: Tapas, paella, and jamón ibérico\n- **Currency**: Euro\n- **Culture**: Known for its vibrant festivals, flamenco music and dance, and rich history influenced by various cultures, including Roman and Moorish.\n- **Geography**: Features diverse landscapes, including beaches, mountains, and plains, with notable regions like Catalonia, Andalusia, and the Basque Country.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'service_tier': 'default'}, id='run--d3ebe06d-497d-43bd-9aef-b0dd1c4d04f5-0')

## RandomBasedExampleSelector

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [16]:
class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples
  
  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables):
    from random import choice

    return [choice(self.examples)]

In [17]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
  examples=examples
)

In [28]:
prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

## Serialization and Composition

In [11]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
  ]
)

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

prompt.format(country="Germmany")

'What is the capital of Germmany'

In [ ]:
chain = prompt | chat

chain.invoke({
  "country" : "Germmany"
})

The capital of Germany is Berlin.

AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'service_tier': 'default'}, id='run--b5f5c7c0-5d33-479e-b9a8-8b74b12b07c6-0')

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ],
)

prompt.format(country="Germmany")

'What is the capital of Germmany'

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

In [46]:
intro = PromptTemplate.from_template(
  """
  You are a role playing assistant.
  And you are impersonating a {character}
  """
)

example = PromptTemplate.from_template(
  """
  this is an example of how you talk:

  Human: {example_question}
  You: {example_answer}
  """
)

start= PromptTemplate.from_template(
  """
  Strat now!

  Human: {question}
  You:
  """
)

final = PromptTemplate.from_template(
  """
  {intro}

  {example}

  {start}
  """
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]

full_prompt = PipelinePromptTemplate(
  final_prompt=final,
  pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_78022/423968787.py:42: LangChainDeprecationWarning: This class is deprecated in favor of chaining individual prompts together.
  full_prompt = PipelinePromptTemplate(


Arrrr, matey! Me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!

AIMessage(content="Arrrr, matey! Me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--68d7e2b3-6f69-4077-a682-01373a5e0d29-0')

## Caching

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1
)

chat.predict("What is the capital of Japan")

/Users/jaylee_83/Documents/_itsjayspace/git_clones/Langchain-7-day-challenge/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chat.predict("What is the capital of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 13,
                "total_tokens": 20,
                "completion_tokens_details": {
             

/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_91718/2341586869.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat.predict("What is the capital of Japan")


'The capital of Japan is Tokyo.'

In [4]:
chat.predict("What is the capital of Korea")

'The capital of South Korea is Seoul. The capital of North Korea is Pyongyang.'

In [5]:
chat.predict("What is the capital of Korea")

'The capital of South Korea is Seoul. The capital of North Korea is Pyongyang.'

## Serialization

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(False) 

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1
)

with get_openai_callback() as usage:
  a = chat.predict("What is the recipt for soju")
  b = chat.predict("What is the recipt for bread")
  

  print(usage)
  print(usage.total_cost) # 전체 비용
  print(usage.total_tokens) # 토큰 총합
  print(usage.prompt_tokens) # 프로프트에 사용된 토튼수
  print(usage.completion_tokens) # 모델이 쓴 토큰

Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')
Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')


Tokens Used: 0
	Prompt Tokens: 0
		Prompt Tokens Cached: 0
	Completion Tokens: 0
		Reasoning Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
0.0
0
0
0
